In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

ERROR: types-requests 2.31.0.20240218 has requirement urllib3>=2, but you'll have urllib3 1.25.8 which is incompatible.
ERROR: kubernetes 29.0.0 has requirement pyyaml>=5.4.1, but you'll have pyyaml 5.3.1 which is incompatible.
ERROR: chromadb 0.4.23 has requirement PyYAML>=6.0.0, but you'll have pyyaml 5.3.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install chromadb

     |████████████████████████████████| 736 kB 492 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install python-dotenv

In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Now you can access the API key as an environment variable
api_key = os.getenv("OPENAI_API_KEY")
langchain = os.getenv("LANGCHAIN_API_KEY")

In [5]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import pysqlite3 as sqlite3
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
loader = PyPDFLoader("../data/Raptor Contract.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(pages):
    return "\n\n".join(doc.page_content for doc in pages)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
rag_chain.invoke("What is D&O tail?")

'D&O tail refers to directors\' and officers\' liability insurance policies that provide coverage for acts or omissions by directors and officers prior to the closing date of a transaction. The company is required to obtain and pay for "tail" insurance policies for the benefit of individuals covered by the existing insurance policy with terms at least as favorable as the current coverage. This provision survives the consummation of the transaction and can be enforced by the D&O indemnitees.'